In [6]:
from PyPDF2 import PdfFileReader
import re
import csv
import fitz
def extract_quiz_sections(pdf_file):
    def extract_text_from_page(page):
        # function to extract text from a page and remove unnecessary whitespaces
        return page.extract_text().strip()

    pdf_sections = []
    current_section = None
    current_content = ""
    in_quiz = False
    current_quiz = None
    current_question = None
    current_options = []
    current_correct_answer = None

        
    for page_num in range(len(pdf_file)):
        text = pdf_file[page_num].get_text("text")

        if text.lower().startswith("section 0"):
            # Save the previous quiz section information, if any
            if current_section is not None:
                pdf_sections.append({
                    "section_number": current_section,
                    "content": current_content,
                    "questions": current_quiz
                    })

            # Extract section number from the text "Section 0X"
            section_number = int(text.lower().split()[1])
            # Iitialize variables for the new section
            current_section = section_number
            current_content = ""
            current_quiz = []
            in_quiz = False
            current_question = None
            current_options = []
            current_correct_answer = None
        elif in_quiz:
                # If inside a quiz section, process the text to extract questions, options, and correct answer
            if re.match(r"^\d+\..+", text):
                    # New question encountered, save the previous question's data
                if current_question is not None:
                    current_quiz.append({
                        "question": current_question,
                        "options": current_options,
                        "correct_answer": current_correct_answer
                    })
                # Initialize variables for the new question
                current_question = text
                current_options = []
                current_correct_answer = None
            elif re.match(r"^[a-h]\..+", text):
                    # Options for the current question (e.g., "a. Option 1")
                current_options.append(text)
            elif "correct answer" in text.lower():
                    # Extracting the correct answer from the text
                current_correct_answer = text.split(":")[-1].strip()
        elif text.lower().startswith("réponses de la section 0"):
                # Indicates the start of the correction section, so we are no longer in a quiz
            in_quiz = False
        elif text.lower().startswith("section"):
                # Indicates the start of a new quiz section, set in_quiz to True
            in_quiz = True
            
            # Accumulate content for the current section
        current_content += text + "\n"

        # Save the last section content and quiz questions
    if current_section is not None:
        pdf_sections.append({
            "section_number": current_section,
            "content": current_content,
            "questions": current_quiz
            })
       
    return pdf_sections

In [7]:
url1= r"C:\Users\USER\Downloads\Le_grand_quiz_de_la_langue_fran_231_aise.pdf"
pdf_file_path = fitz.open(url1)
sections = extract_quiz_sections(pdf_file_path)
print(sections)

[{'section_number': 1, 'content': 'section 01\nS’amuser à conjuguer? Pourquoi pas?\n\n \n1.\nUn de ces verbes n’est pas à l’imparfait. Lequel?\na.\nje bondissais\nb.\nil s’élançait\nc.\nnous courrions\nd.\nelles accéléraient\n2.\nComplétez le verbe dans la phrase suivante.\nIl a trouvé ce qu’il cherchait en navig __________ sur Internet.\n3.\nÀ la 3e personne du pluriel, tous les verbes conjugués à un\nmode personnel se terminent toujours par -nt, quels que\nsoient le temps et le mode: vrai ou faux?\n4.\nTous ces verbes sont du 2e groupe, sauf un. Quel est l’intrus?\na.\nagir\nb.\ndéglutir\nc.\nhaïr\nd.\nlanguir\ne.\npervertir\nf.\npressentir\ng.\nsertir\nh.\nvrombir\n5.\nÀ quel temps est conjugué le verbe de la phrase suivante?\nIl est assis dans son fauteuil.\n \n6.\nDonnez la 3e personne du pluriel du verbe élire au passé simple.\n \n7.\nSoulignez la ou les erreurs de conjugaison du conseil suivant.\nNe disez pas «disez», disez «dites».\n8.\nCombien y a-t-il de modes personnels?\n\n

In [8]:
def split_content(sections):

    for section in sections:
         section["content"]=section["content"].split("\n")
    return sections           

In [9]:
sections=split_content(sections)

In [10]:
regex2 = re.compile('([0-9]+\.)')
regex = re.compile('([a-z]\.)')

def concat_content(listefile2):
    i=0
    result=[]
    while(i<len(listefile2)-1):
        if(regex2.search(listefile2[i]) and len(listefile2[i].strip())<5):
            result.append(listefile2[i]+" "+listefile2[i+1])
            i+=2
            
        elif(regex.search(listefile2[i]) and len(listefile2[i].strip())<5):
        
            result.append(listefile2[i].capitalize()+" "+listefile2[i+1]) 
            i+=2
        else:
            result.append(listefile2[i])  
            i+=1
    return  result 

In [11]:
def concatenation(sections):

    for section in sections:
         section["content"]=concat_content(section["content"])
    return sections 

In [12]:
sections=concatenation(sections)
print(sections)

[{'section_number': 1, 'content': ['section 01', 'S’amuser à conjuguer? Pourquoi pas?', '', ' ', '1. Un de ces verbes n’est pas à l’imparfait. Lequel?', 'A. je bondissais', 'B. il s’élançait', 'C. nous courrions', 'D. elles accéléraient', '2. Complétez le verbe dans la phrase suivante.', 'Il a trouvé ce qu’il cherchait en navig __________ sur Internet.', '3. À la 3e personne du pluriel, tous les verbes conjugués à un', 'mode personnel se terminent toujours par -nt, quels que', 'soient le temps et le mode: vrai ou faux?', '4. Tous ces verbes sont du 2e groupe, sauf un. Quel est l’intrus?', 'A. agir', 'B. déglutir', 'C. haïr', 'D. languir', 'E. pervertir', 'F. pressentir', 'G. sertir', 'H. vrombir', '5. À quel temps est conjugué le verbe de la phrase suivante?', 'Il est assis dans son fauteuil.', ' ', '6. Donnez la 3e personne du pluriel du verbe élire au passé simple.', ' ', '7. Soulignez la ou les erreurs de conjugaison du conseil suivant.', 'Ne disez pas «disez», disez «dites».', '8. 

In [9]:
def clean_list(listline):

    for x in listline:
        if (len(x)<2):
            listline.remove(x)
    return listline 

In [10]:
def phrases_incomplètes(result):

    lines=[]
    for i in range (len(result)):
            line=result[i]
            j=i+1
            if(j<len(result) and len(result[j])!=0):
                while (j<len(result)and (result[j].islower()) ):  
                      line=line.strip()+" "+result[j].strip()
                      result[j]=""
                      j=j+1
                     
            
            lines.append(line)
    lines=clean_list(lines)
    return lines


In [11]:
def application_phrases_inc(sections):

    for section in sections:
         section["content"]=phrases_incomplètes(section["content"])
    return sections 

In [12]:
sections=application_phrases_inc(sections)

In [13]:
print(sections)

[{'section_number': 1, 'content': ['section 01', 'S’amuser à conjuguer? Pourquoi pas?', '1. Un de ces verbes n’est pas à l’imparfait. Lequel?', 'A. je bondissais', 'B. il s’élançait', 'C. nous courrions', 'D. elles accéléraient', '2. Complétez le verbe dans la phrase suivante.', 'Il a trouvé ce qu’il cherchait en navig __________ sur Internet.', '3. À la 3e personne du pluriel, tous les verbes conjugués à un mode personnel se terminent toujours par -nt, quels que soient le temps et le mode: vrai ou faux?', '4. Tous ces verbes sont du 2e groupe, sauf un. Quel est l’intrus?', 'A. agir', 'B. déglutir', 'C. haïr', 'D. languir', 'E. pervertir', 'F. pressentir', 'G. sertir', 'H. vrombir', '5. À quel temps est conjugué le verbe de la phrase suivante?', 'Il est assis dans son fauteuil.', '6. Donnez la 3e personne du pluriel du verbe élire au passé simple.', '7. Soulignez la ou les erreurs de conjugaison du conseil suivant.', 'Ne disez pas «disez», disez «dites».', '8. Combien y a-t-il de modes

In [14]:
def extract_correction(sections):
    for section in sections:
        L = []
        for i in range(len(section["content"])):
            if section["content"][i].startswith("Réponses de la section"):
                L = section["content"][i:]
                section["questions"] = L.copy()
                break  # Exit the loop after finding the "questions" content
    return sections

In [15]:
sections=extract_correction(sections)

In [16]:
print(sections)

[{'section_number': 1, 'content': ['section 01', 'S’amuser à conjuguer? Pourquoi pas?', '1. Un de ces verbes n’est pas à l’imparfait. Lequel?', 'A. je bondissais', 'B. il s’élançait', 'C. nous courrions', 'D. elles accéléraient', '2. Complétez le verbe dans la phrase suivante.', 'Il a trouvé ce qu’il cherchait en navig __________ sur Internet.', '3. À la 3e personne du pluriel, tous les verbes conjugués à un mode personnel se terminent toujours par -nt, quels que soient le temps et le mode: vrai ou faux?', '4. Tous ces verbes sont du 2e groupe, sauf un. Quel est l’intrus?', 'A. agir', 'B. déglutir', 'C. haïr', 'D. languir', 'E. pervertir', 'F. pressentir', 'G. sertir', 'H. vrombir', '5. À quel temps est conjugué le verbe de la phrase suivante?', 'Il est assis dans son fauteuil.', '6. Donnez la 3e personne du pluriel du verbe élire au passé simple.', '7. Soulignez la ou les erreurs de conjugaison du conseil suivant.', 'Ne disez pas «disez», disez «dites».', '8. Combien y a-t-il de modes

In [17]:

def extract_correction_step2(liste_dictionnaires):
    L=[]
    for index, dictionnaire in enumerate(liste_dictionnaires):
        if dictionnaire["content"][index].startswith("Réponses de la section 0"):
            liste_dictionnaires[index]["questions"] = dictionnaire["content"]
    L=liste_dictionnaires[-1]["questions"]
    return L

# Exemple d'utilisation :
L=extract_correction_step2(sections)

In [18]:
def split_sections_correction(entree):
    sections = []
    current_section = []

    for line in entree:
        if line.startswith("Réponses de la section"):
            if current_section:
                sections.append(current_section)
                current_section = []
        current_section.append(line)

    if current_section:
        sections.append(current_section)

    return sections


sections_correction = split_sections_correction(L)
print(sections_correction)

[['Réponses de la section 01', 'S’amuser à conjuguer? Pourquoi pas?', ' ', '1. c. Nous courrions est au conditionnel présent.', '2. Il a trouvé ce qu’il cherchait en naviguant sur Internet. (Les verbes en -guer gardent le u dans toute la conjugaison.)', '3. Vrai. Il n’y a aucune exception à cette règle.', '4. f. Pressentir. (Verbe du 3e groupe: nous pressentons.)', '5. Au présent de l’indicatif: la phrase est construite au passif. Au passé composé, on aurait «il a été assis».', '6. Ils/elles élurent. Élire se conjugue sur le modèle de lire et non de dire.', '7. Dites est la seule forme correcte. Ne dites pas «disez», dites «dites»».', '8. Quatre: l’indicatif, le subjonctif, l’impératif et le conditionnel. 9. a.', '10. Plus vous vous entraînerez, plus vous acquerrez d’autonomie.', '11. Oui. Il n’y a pas de s à la 2e personne du singulier du présent de l’impératif (contrairement à la 2e personne du singulier du présent de l’indicatif).', '12. Il faut que je lui envoie mes résultats ou qu

In [19]:
def sections_correction_update(sections,sections_correction):
    for i in range(len(sections_correction)):
        sections[i]["questions"]=sections_correction[i]
    return sections
sections_correction_questions=sections_correction_update(sections,sections_correction)
print(sections_correction_questions)

[{'section_number': 1, 'content': ['section 01', 'S’amuser à conjuguer? Pourquoi pas?', '1. Un de ces verbes n’est pas à l’imparfait. Lequel?', 'A. je bondissais', 'B. il s’élançait', 'C. nous courrions', 'D. elles accéléraient', '2. Complétez le verbe dans la phrase suivante.', 'Il a trouvé ce qu’il cherchait en navig __________ sur Internet.', '3. À la 3e personne du pluriel, tous les verbes conjugués à un mode personnel se terminent toujours par -nt, quels que soient le temps et le mode: vrai ou faux?', '4. Tous ces verbes sont du 2e groupe, sauf un. Quel est l’intrus?', 'A. agir', 'B. déglutir', 'C. haïr', 'D. languir', 'E. pervertir', 'F. pressentir', 'G. sertir', 'H. vrombir', '5. À quel temps est conjugué le verbe de la phrase suivante?', 'Il est assis dans son fauteuil.', '6. Donnez la 3e personne du pluriel du verbe élire au passé simple.', '7. Soulignez la ou les erreurs de conjugaison du conseil suivant.', 'Ne disez pas «disez», disez «dites».', '8. Combien y a-t-il de modes

In [20]:
def extraction_qcm(sections):
    regex = re.compile('(([A-Z]+\.|À\.)\s*[A-Za-z])')
    questions = []
    current_question = None
    current_correction = ""

    for section in sections:
        for j in range(len(section["content"])):
            if regex2.search(section["content"][j]):
                if current_question and current_question['options']:
                    questions.append(current_question)

                for i in range(len(section["questions"])):
                    if section["questions"][i].split(".")[0].strip() == section["content"][j].split(".")[0].strip():
                        current_correction = section["questions"][i]

                current_question = {'question': section["content"][j].strip(), 'options': [], 'correction': current_correction}
            elif current_question and regex.search(section["content"][j]):
                # Add the options of the current question
                current_question['options'].append(section["content"][j].strip())

    # Check if the last question has options and add it to the list
    if current_question and current_question['options']:
        questions.append(current_question)

    return questions

In [21]:
data = extraction_qcm(sections)
print(data)

[{'question': '1. Un de ces verbes n’est pas à l’imparfait. Lequel?', 'options': ['A. je bondissais', 'B. il s’élançait', 'C. nous courrions', 'D. elles accéléraient'], 'correction': '1. c. Nous courrions est au conditionnel présent.'}, {'question': '4. Tous ces verbes sont du 2e groupe, sauf un. Quel est l’intrus?', 'options': ['A. agir', 'B. déglutir', 'C. haïr', 'D. languir', 'E. pervertir', 'F. pressentir', 'G. sertir', 'H. vrombir'], 'correction': '4. f. Pressentir. (Verbe du 3e groupe: nous pressentons.)'}, {'question': '9. Comment s’appelle le temps formé avec l’auxiliaire à l’imparfait de l’indicatif et le participe passé?', 'options': ['A. le plus-que-parfait', 'B. le plus-qu’imparfait', 'C. le passé de l’imparfait'], 'correction': '8. Quatre: l’indicatif, le subjonctif, l’impératif et le conditionnel. 9. a.'}, {'question': '13. Quelle est la forme correcte du verbe bouillir à l’imparfait du subjonctif?', 'options': ['A. qu’il bouillît', 'B. qu’il bouillisse', 'C. qu’il bouill

In [22]:
# creation of the CSV file
filename = "quiz_data.csv"
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    fieldnames = ['question', 'option a', 'option b', 'option c', 'option d', 'option e','option f', 'option g', 'option h', 'option i','correct_answer', 'explanation']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for question in data:
        
        options = question['options'] + [''] * (9 - len(question['options']))  
        writer.writerow({
            'question': question['question'],
            'option a': options[0],
            'option b': options[1],
            'option c': options[2],
            'option d': options[3],
            'option e': options[4],
            'option f': options[5],
            'option g': options[6],
            'option h': options[7],
            'option i': options[8],
            

            'correct_answer': question['correction'],
            'explanation': ''
        })

print("CSV file has been created successfully.")

CSV file has been created successfully.


CSV file has been created successfully.


In [26]:


# Verify data for each section
for i, section in enumerate(sections, start=1):
    print(f"Section {i}:")
    data_section = extraction_qcm([section])  # Pass the current section as a list
    for question_data in data_section:
        print("*****", question_data['question'])
        for option in question_data['options']:
            print(option)
        print("correction", question_data['correction'])
    print("==========================")



Section 1:
***** 1. Un de ces verbes n’est pas à l’imparfait. Lequel?
A. je bondissais
B. il s’élançait
C. nous courrions
D. elles accéléraient
correction 1. c. Nous courrions est au conditionnel présent.
***** 4. Tous ces verbes sont du 2e groupe, sauf un. Quel est l’intrus?
A. agir
B. déglutir
C. haïr
D. languir
E. pervertir
F. pressentir
G. sertir
H. vrombir
correction 4. f. Pressentir. (Verbe du 3e groupe: nous pressentons.)
***** 9. Comment s’appelle le temps formé avec l’auxiliaire à l’imparfait de l’indicatif et le participe passé?
A. le plus-que-parfait
B. le plus-qu’imparfait
C. le passé de l’imparfait
correction 8. Quatre: l’indicatif, le subjonctif, l’impératif et le conditionnel. 9. a.
***** 13. Quelle est la forme correcte du verbe bouillir à l’imparfait du subjonctif?
A. qu’il bouillît
B. qu’il bouillisse
C. qu’il bouillusse
correction 13. a. À l’imparfait du subjonctif, il y a toujours un accent circonflexe à la 3e personne du singulier.
***** 15. Quelle est la forme cor

***** 1. La flavescence, c’est:
A. une odeur
B. une texture
C. une couleur
correction 1. c. Ce qui est flavescent est blond, doré. Ne pas confondre avec flaveur, qui est la perception simultanée de l’odeur et du goût d’un aliment. 2. a. une fièvre de cheval b. une faim de loup c. une langue de vipère d. des larmes de crocodile 3. bourdalou (ruban à chapeau) – galure – béret – licorne (mais le bicorne est un chapeau) – bolivar – panama – brodequin (chaussure) – myrte (mais la mitre est une coiffure) – kippa – chapka – cabriolet – tiare – paluche (main)
***** 2. Complétez les expressions avec un nom d’animal.
A. une fièvre de __________
B. une faim de __________
C. une langue de __________
D. des larmes de __________
correction 1. c. Ce qui est flavescent est blond, doré. Ne pas confondre avec flaveur, qui est la perception simultanée de l’odeur et du goût d’un aliment. 2. a. une fièvre de cheval b. une faim de loup c. une langue de vipère d. des larmes de crocodile 3. bourdalou (ruban à

***** 4. Comment écrit-on le premier mot du proverbe … beau mentir qui vient de loin?
A. A
correction 4. a. Il s’agit du verbe avoir; le proverbe signifie qu’il est facile de mentir quand ce qu’on dit ne peut pas être vérifié.
***** 6. Le proverbe À vaincre sans péril, on triomphe sans gloire est une citation de:
A. Corneille
B. Hugo
C. Racine
correction 6. a. Il s’agit d’une réplique du Comte s’adressant à Don Rodrigue,
***** 8. Quelle est la nature grammaticale de bien dans le proverbe Bien mal acquis ne profite jamais?
A. adjectif
B. adverbe
C. nom
correction 8. c. Bien est ici un nom masculin et a le sens de «un bien qui a été acquis de façon malhonnête n’est jamais vraiment avantageux».
***** 11. D’après le proverbe, à qui la chance sourit-elle?
A. aux audacieux
B. aux courageux
C. aux fortunés
correction 11. a. Seuls ceux qui ont l’audace d’entreprendre quelque chose de risqué peuvent espérer réussir.
***** 20. Quel proverbe incite à ne pas tomber dans la procrastination?
A. C’es